# Data Manipulation and Visualization - Food Project

### Goal
The goal is to analyze the environmental impact of global food and feed production and distribution.

### Requirements
The project needs to be divided into six phases:
- Discovery
- Data Selection
- Data Cleaning
- Data Transformation
- Data Exploration
- Data Visualization

The project should be delivered with the awareness that the notebook must be similar to a presentation to be shown to a hypothetical client.

# Discovery

### Problem Identification
Our global population is forecasted to grow from today's 7.3 billion to 9.7 billion by the year 2050. Finding solutions to feed the growing world population has become a hot topic for agriculture and food organizations, entrepreneurs, and philanthropists.

These solutions range from changing how we cultivate and raise our food to how we eat.

To make matters more challenging, the Earth's climate is changing and is influenced by agriculture. As the world population has expanded and become wealthier, the demand for food, energy, and water has seen a rapid increase.

The demand has not only increased for all these three, but they are also heavily interconnected: food production requires water and energy; traditional energy production requires water resources; agriculture provides a potential source of energy and needs a significant amount of water.

Ensuring that everyone worldwide has access to a nutritious and sustainable diet is one of the major challenges we must face.

### Aims
With this project, we aim to answer the following questions:
- What has been the trend in food and feed over the past years?
- Which foods and feeds were mostly produced at the beginning and by the end of our analysis period?
- How had the production of food and feed varied in the most productive country by the end of the analysis period?
- Which foods were responsible for the highest emissions?
- Which foods used the most of the land?

# Data Selection

The following datasets have been used:
- [FAO.csv](https://www.kaggle.com/dorbicycle/world-foodfeed-production):
    - Provides an overview of global food production
    - Focuses on comparing food produced for human consumption and feed produced for animals
    - Sourced from the United Nations Food and Agriculture Organization
    - Offers data for over 245 countries and territories from 1961 to the most recent update (2013)

- [Food_Production.csv](https://www.kaggle.com/selfvivek/environment-impact-of-food-production):
    - Contains information on the 43 most common foods grown worldwide
    - Includes 23 columns describing values such as water usage and greenhouse gas emissions

- [Population.csv](https://data.worldbank.org/indicator/SP.POP.TOTL?most_recent_year_desc=true):
    - Contains data about world population by country from 1960 to the most recent update (2022)

- [Taiwan.csv](https://www.worldometers.info/world-population/taiwan-population/)
    - Contains data about taiwan population from 1961 to 2013 (analysis period)

- [Temperature.csv](https://www.kaggle.com/datasets/sevgisarac/temperature-change/data)
    - Contains information on average variations in surface temperature by country, with annual updates spanning the period 1961-2019.
    - These values recorded in the dataset are understood as anomalies, representing temperature variations relative to a baseline climatology corresponding to the period 1951-1980

Using various datasets allows for a more in-depth analysis. The most important variables emerging from the selected datasets are:
- Data on food and feed production and their impact on the environment
- Data on the world population
- Data on changes in the Earth's temperature

As we will see during the analysis, these three elements are interconnected.

### Import Libraries and Datasets

In [1]:
# libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px
import plotly.graph_objects as go
import country_converter as coco
from difflib import SequenceMatcher
from raceplotly.plots import barplot
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression
# settings
pd.set_option('display.max_columns', None)

In [2]:
# import csv
df_fao=pd.read_csv('csv/FAO.csv',encoding='latin-1')
df_prod=pd.read_csv('csv/Food_Production.csv')
df_pop=pd.read_csv('csv/Population.csv')
df_pop_taiwan=pd.read_csv('csv/Taiwan.csv')
df_temp=pd.read_csv('csv/Temperature.csv',encoding='latin-1')

# Data Cleaning & Data Transformation

### FAO dataset

In [3]:
# checking what the dataset looks like
df_fao.info()
df_fao.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21477 entries, 0 to 21476
Data columns (total 63 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area Abbreviation  21477 non-null  object 
 1   Area Code          21477 non-null  int64  
 2   Area               21477 non-null  object 
 3   Item Code          21477 non-null  int64  
 4   Item               21477 non-null  object 
 5   Element Code       21477 non-null  int64  
 6   Element            21477 non-null  object 
 7   Unit               21477 non-null  object 
 8   latitude           21477 non-null  float64
 9   longitude          21477 non-null  float64
 10  Y1961              17938 non-null  float64
 11  Y1962              17938 non-null  float64
 12  Y1963              17938 non-null  float64
 13  Y1964              17938 non-null  float64
 14  Y1965              17938 non-null  float64
 15  Y1966              17938 non-null  float64
 16  Y1967              179

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


Observing the dataset, we begin to perform the initial cleaning.

In [4]:
# drop columns from the DataFrame
df_fao.drop(columns=['Area Code', 'Item Code', 'Element Code', 'Unit'], inplace=True)

# rename columns containing 'Y' by removing the 'Y' character
df_fao.rename(columns={x: x[1:] for x in df_fao.columns if 'Y' in x}, inplace=True)

# rename the column 'Area Abbreviation' to 'Area Code'
df_fao.rename(columns={'Area Abbreviation': 'Area Code'}, inplace=True)

# rename columns by replacing spaces with underscores and converting to lowercase
df_fao.rename(columns={i: i.replace(' ', '_').lower() for i in df_fao.columns}, inplace=True)

# remove duplicate rows from the DataFrame
df_fao = df_fao.drop_duplicates()

Let's determine the number of NaN values for each column.

In [5]:
# check NaN values
for column in df_fao.columns:
    num_nan = df_fao[column].isna().sum()
    percentage_nan = (num_nan / len(df_fao)) * 100
    print(f"Column '{column}': {percentage_nan:.2f}% NaN")

Column 'area_code': 0.00% NaN
Column 'area': 0.00% NaN
Column 'item': 0.00% NaN
Column 'element': 0.00% NaN
Column 'latitude': 0.00% NaN
Column 'longitude': 0.00% NaN
Column '1961': 16.43% NaN
Column '1962': 16.43% NaN
Column '1963': 16.43% NaN
Column '1964': 16.43% NaN
Column '1965': 16.43% NaN
Column '1966': 16.43% NaN
Column '1967': 16.43% NaN
Column '1968': 16.43% NaN
Column '1969': 16.43% NaN
Column '1970': 16.43% NaN
Column '1971': 16.43% NaN
Column '1972': 16.43% NaN
Column '1973': 16.43% NaN
Column '1974': 16.43% NaN
Column '1975': 16.43% NaN
Column '1976': 16.43% NaN
Column '1977': 16.43% NaN
Column '1978': 16.43% NaN
Column '1979': 16.43% NaN
Column '1980': 16.43% NaN
Column '1981': 16.43% NaN
Column '1982': 16.43% NaN
Column '1983': 16.43% NaN
Column '1984': 16.43% NaN
Column '1985': 16.43% NaN
Column '1986': 16.43% NaN
Column '1987': 16.43% NaN
Column '1988': 16.43% NaN
Column '1989': 16.43% NaN
Column '1990': 15.85% NaN
Column '1991': 15.85% NaN
Column '1992': 4.59% NaN
Co

The production data columns between 1961 and 1991 exhibit the highest percentages of missing values.

Let's identify the countries involved to gain further insight into the dataset.

In [6]:
# print unique values of the 'area' column that have at least one NaN value
print(df_fao.loc[df_fao.loc[:, '1961':'1991'].isnull().any(axis=1), 'area'].unique().tolist())

['Armenia', 'Azerbaijan', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Croatia', 'Czechia', 'Estonia', 'Ethiopia', 'Georgia', 'Kazakhstan', 'Kyrgyzstan', 'Latvia', 'Lithuania', 'Luxembourg', 'Montenegro', 'Oman', 'Republic of Moldova', 'Russian Federation', 'Serbia', 'Slovakia', 'Slovenia', 'Sudan', 'Tajikistan', 'The former Yugoslav Republic of Macedonia', 'Turkmenistan', 'Ukraine', 'Uzbekistan']


We can notice that most of the nations with at least one NaN value belonged to the USSR or Yugoslavia.

Therefore, supported also by the dataset documentation, the main cause of data loss is due to geopolitical changes.

Deleting lines with missing values increases the risk of data loss. In this context it seems acceptable to remove those rows affected by geopolitical changes to obtain a more accurate analysis of the dataset.

In [7]:
df_fao = df_fao.dropna()

The rows with all null production values are also eliminated as they are irrelevant for the analysis.

In [8]:
# keep only the rows where all the data are different from zero
df_fao = df_fao.loc[~(df_fao.loc[:, '1961':] == 0).all(axis=1)]
df_fao.reset_index(drop=True,inplace=True)

The function below checks the similarity between two words. This function is useful to identify any typos in the dataset or suspicious situations that need to be manually checked. It returns a table with the names and the percentage of similarity.

In [9]:
# function to check similarity between names
def check_similarity(names):
    results={} # to store results

    for name1 in names:
        similarities={} # to store similarities between name1 and other names
        # compare name1 with every other name in the list
        for name2 in names:
            # calculate similarity score between name1 and name2 and round it to 2 decimal place
            similarities[name2]=round(SequenceMatcher(None,name1,name2).ratio(),2)
        # sort similarities for name1 with other names in descending order based on similarity score
        results[name1]=sorted(similarities.items(),key=lambda x: x[1], reverse=True)
        
    # convert the results dictionary into a pandas DataFrame and return
    return pd.DataFrame(results)

The function is executed for the `area` and `item` columns.

In [10]:
check_similarity(df_fao.area.unique()).head(3)

,Afghanistan,Albania,Algeria,Angola,Antigua and Barbuda,Argentina,Australia,Austria,Bahamas,Bangladesh,Barbados,Belize,Benin,Bermuda,Bolivia (Plurinational State of),Botswana,Brazil,Brunei Darussalam,Bulgaria,Burkina Faso,Cabo Verde,Cambodia,Cameroon,Canada,Central African Republic,Chad,Chile,"China, Hong Kong SAR","China, Macao SAR","China, mainland","China, Taiwan Province of",Colombia,Congo,Costa Rica,Côte d'Ivoire,Cuba,Cyprus,Democratic People's Republic of Korea,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Fiji,Finland,France,French Polynesia,Gabon,Gambia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hungary,Iceland,India,Indonesia,Iran (Islamic Republic of),Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kenya,Kiribati,Kuwait,Lao People's Democratic Republic,Lebanon,Lesotho,Liberia,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Mauritania,Mauritius,Mexico,Mongolia,Morocco,Mozambique,Myanmar,Namibia,Nepal,Netherlands,New Caledonia,New Zealand,Nicaragua,Niger,Nigeria,Norway,Pakistan,Panama,Paraguay,Peru,Philippines,Poland,Portugal,Republic of Korea,Romania,Rwanda,Saint Kitts and Nevis,Saint Lucia,Saint Vincent and the Grenadines,Samoa,Sao Tome and Principe,Saudi Arabia,Senegal,Sierra Leone,Solomon Islands,South Africa,Spain,Sri Lanka,Suriname,Swaziland,Sweden,Switzerland,Thailand,Timor-Leste,Togo,Trinidad and Tobago,Tunisia,Turkey,Uganda,United Arab Emirates,United Kingdom,United Republic of Tanzania,United States of America,Uruguay,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Yemen,Zambia,Zimbabwe
0,"(Afghanistan, 1.0)","(Albania, 1.0)","(Algeria, 1.0)","(Angola, 1.0)","(Antigua and Barbuda, 1.0)","(Argentina, 1.0)","(Australia, 1.0)","(Austria, 1.0)","(Bahamas, 1.0)","(Bangladesh, 1.0)","(Barbados, 1.0)","(Belize, 1.0)","(Benin, 1.0)","(Bermuda, 1.0)","(Bolivia (Plurinational State of), 1.0)","(Botswana, 1.0)","(Brazil, 1.0)","(Brunei Darussalam, 1.0)","(Bulgaria, 1.0)","(Burkina Faso, 1.0)","(Cabo Verde, 1.0)","(Cambodia, 1.0)","(Cameroon, 1.0)","(Canada, 1.0)","(Central African Republic, 1.0)","(Chad, 1.0)","(Chile, 1.0)","(China, Hong Kong SAR, 1.0)","(China, Macao SAR, 1.0)","(China, mainland, 1.0)","(China, Taiwan Province of, 1.0)","(Colombia, 1.0)","(Congo, 1.0)","(Costa Rica, 1.0)","(Côte d'Ivoire, 1.0)","(Cuba, 1.0)","(Cyprus, 1.0)","(Democratic People's Republic of Korea, 1.0)","(Denmark, 1.0)","(Djibouti, 1.0)","(Dominica, 1.0)","(Dominican Republic, 1.0)","(Ecuador, 1.0)","(Egypt, 1.0)","(El Salvador, 1.0)","(Fiji, 1.0)","(Finland, 1.0)","(France, 1.0)","(French Polynesia, 1.0)","(Gabon, 1.0)","(Gambia, 1.0)","(Germany, 1.0)","(Ghana, 1.0)","(Greece, 1.0)","(Grenada, 1.0)","(Guatemala, 1.0)","(Guinea, 1.0)","(Guinea-Bissau, 1.0)","(Guyana, 1.0)","(Haiti, 1.0)","(Honduras, 1.0)","(Hungary, 1.0)","(Iceland, 1.0)","(India, 1.0)","(Indonesia, 1.0)","(Iran (Islamic Republic of), 1.0)","(Iraq, 1.0)","(Ireland, 1.0)","(Israel, 1.0)","(Italy, 1.0)","(Jamaica, 1.0)","(Japan, 1.0)","(Jordan, 1.0)","(Kenya, 1.0)","(Kiribati, 1.0)","(Kuwait, 1.0)","(Lao People's Democratic Republic, 1.0)","(Lebanon, 1.0)","(Lesotho, 1.0)","(Liberia, 1.0)","(Madagascar, 1.0)","(Malawi, 1.0)","(Malaysia, 1.0)","(Maldives, 1.0)","(Mali, 1.0)","(Malta, 1.0)","(Mauritania, 1.0)","(Mauritius, 1.0)","(Mexico, 1.0)","(Mongolia, 1.0)","(Morocco, 1.0)","(Mozambique, 1.0)","(Myanmar, 1.0)","(Namibia, 1.0)","(Nepal, 1.0)","(Netherlands, 1.0)","(New Caledonia, 1.0)","(New Zealand, 1.0)","(Nicaragua, 1.0)","(Niger, 1.0)","(Nigeria, 1.0)","(Norway, 1.0)","(Pakistan, 1.0)","(Panama, 1.0)","(Paraguay, 1.0)","(Peru, 1.0)","(Philippines, 1.0)","(Poland, 1.0)","(Portugal, 1.0)","(Republic of Korea, 1.0)","(Romania, 1.0)","(Rwanda, 1.0)","(Saint Kitts and Nevis, 1.0)","(Saint Lucia, 1.0)","(Saint Vincent and the Grenadines, 1.0)","(Samoa, 1.0)","(Sao Tome and Principe, 1.0)","(Saudi Arabia, 1.0)","(Senegal, 1.0)","(Sierra Leone, 1.0)","(Solomon Islands, 1.0)","(South Africa, 1.0)","(Spain, 1

In [11]:
check_similarity(df_fao.item.unique()).head(3)

,Wheat and products,Rice (Milled Equivalent),Barley and products,Maize and products,Millet and products,"Cereals, Other",Potatoes and products,Sugar cane,Sugar beet,Sugar (Raw Equivalent),"Sweeteners, Other",Honey,"Pulses, Other and products",Nuts and products,Coconuts - Incl Copra,Sesame seed,Olives (including preserved),Soyabean Oil,Sunflowerseed Oil,Rape and Mustard Oil,Cottonseed Oil,Palm Oil,Sesameseed Oil,Olive Oil,"Oilcrops Oil, Other",Tomatoes and products,"Vegetables, Other","Oranges, Mandarines","Citrus, Other",Bananas,Apples and products,Pineapples and products,Dates,Grapes and products (excl wine),"Fruits, Other",Coffee and products,Cocoa Beans and products,Tea (including mate),"Spices, Other",Wine,Beer,"Beverages, Alcoholic",Bovine Meat,Mutton & Goat Meat,Poultry Meat,"Meat, Other","Offals, Edible","Butter, Ghee",Cream,"Fats, Animals, Raw",Eggs,Milk - Excluding Butter,Freshwater Fish,Infant food,Cereals - Excluding Beer,Starchy Roots,Sugar Crops,Sugar & Sweeteners,Pulses,Treenuts,Oilcrops,Vegetable Oils,Vegetables,Fruits - Excluding Wine,Stimulants,Spices,Alcoholic Beverages,Meat,Offals,Animal fats,"Fish, Seafood",Miscellaneous,Rye and products,Oats,Sorghum and products,Sweet potatoes,Beans,Groundnuts (Shelled Eq),"Oilcrops, Other",Groundnut Oil,Maize Germ Oil,Onions,"Lemons, Limes and products",Grapefruit and products,Plantains,Pimento,"Beverages, Fermented",Pigmeat,Demersal Fish,Pelagic Fish,"Marine Fish, Other",Crustaceans,Cephalopods,"Molluscs, Other",Peas,Soyabeans,Rape and Mustardseed,Coconut Oil,Pepper,Cassava and products,Palmkernel Oil,"Fish, Body Oil","Roots, Other","Fish, Liver Oil","Aquatic Animals, Others","Aquatic Products, Other",Sunflower seed,Sugar non-centrifugal,Ricebran Oil,Cloves,Yams,Palm kernels,Cottonseed,Aquatic Plants
0,"(Wheat and products, 1.0)","(Rice (Milled Equivalent), 1.0)","(Barley and products, 1.0)","(Maize and products, 1.0)","(Millet and products, 1.0)","(Cereals, Other, 1.0)","(Potatoes and products, 1.0)","(Sugar cane, 1.0)","(Sugar beet, 1.0)","(Sugar (Raw Equivalent), 1.0)","(Sweeteners, Other, 1.0)","(Honey, 1.0)","(Pulses, Other and products, 1.0)","(Nuts and products, 1.0)","(Coconuts - Incl Copra, 1.0)","(Sesame seed, 1.0)","(Olives (including preserved), 1.0)","(Soyabean Oil, 1.0)","(Sunflowerseed Oil, 1.0)","(Rape and Mustard Oil, 1.0)","(Cottonseed Oil, 1.0)","(Palm Oil, 1.0)","(Sesameseed Oil, 1.0)","(Olive Oil, 1.0)","(Oilcrops Oil, Other, 1.0)","(Tomatoes and products, 1.0)","(Vegetables, Other, 1.0)","(Oranges, Mandarines, 1.0)","(Citrus, Other, 1.0)","(Bananas, 1.0)","(Apples and products, 1.0)","(Pineapples and products, 1.0)","(Dates, 1.0)","(Grapes and products (excl wine), 1.0)","(Fruits, Other, 1.0)","(Coffee and products, 1.0)","(Cocoa Beans and products, 1.0)","(Tea (including mate), 1.0)","(Spices, Other, 1.0)","(Wine, 1.0)","(Beer, 1.0)","(Beverages, Alcoholic, 1.0)","(Bovine Meat, 1.0)","(Mutton & Goat Meat, 1.0)","(Poultry Meat, 1.0)","(Meat, Other, 1.0)","(Offals, Edible, 1.0)","(Butter, Ghee, 1.0)","(Cream, 1.0)","(Fats, Animals, Raw, 1.0)","(Eggs, 1.0)","(Milk - Excluding Butter, 1.0)","(Freshwater Fish, 1.0)","(Infant food, 1.0)","(Cereals - Excluding Beer, 1.0)","(Starchy Roots, 1.0)","(Sugar Crops, 1.0)","(Sugar & Sweeteners, 1.0)","(Pulses, 1.0)","(Treenuts, 1.0)","(Oilcrops, 1.0)","(Vegetable Oils, 1.0)","(Vegetables, 1.0)","(Fruits - Excluding Wine, 1.0)","(Stimulants, 1.0)","(Spices, 1.0)","(Alcoholic Beverages, 1.0)","(Meat, 1.0)","(Offals, 1.0)","(Animal fats, 1.0)","(Fish, Seafood, 1.0)","(Miscellaneous, 1.0)","(Rye and products, 1.0)","(Oats, 1.0)","(Sorghum and products, 1.0)","(Sweet potatoes, 1.0)","(Beans, 1.0)","(Groundnuts (Shelled Eq), 1.0)","(Oilcrops, Other, 1.0)","(Groundnut Oil, 1.0)","(Maize Germ Oil, 1.0)","(Onions, 1.0)","(Lemons, Limes and products, 1.0)","(Grapefruit and products, 1.0)","(Plantains, 1.0)","(Pimento, 1.0)","(Beverages, Fermented, 1.0)","(Pigmeat, 1.0)","(Demersal Fish, 1.0)","(Pelagic Fish, 1.0)","(Marine

Considering the output of the function, we make the following modifications to the dataset.

In [12]:
df_fao.replace({'area': {"C�te d'Ivoire": "Côte d'Ivoire", 'China, Hong Kong SAR':'Hong Kong',
                         'China, Macao SAR':'Macao', 'China, Taiwan Province of':'Taiwan',
                         'China, mainland':'China', 'Bolivia (Plurinational State of)':'Bolivia',
                         'Brunei Darussalam':'Brunei', 'Iran (Islamic Republic of)':'Iran',
                         'Venezuela (Bolivarian Republic of)':'Venezuela'}, 
                'item': {'Beverages, Alcoholic': 'Alcoholic Beverages'}}, 
               inplace=True)

Now we check if multiple elements in the `area` column have been linked to an element in the `area_abbreviation` column.

In [13]:
# group by 'area_abbreviation' and get all unique values associated in the 'area' column
area_ab_grouped = df_fao.groupby('area_code')['area'].unique()

# filter values that have more than one association
print(area_ab_grouped[area_ab_grouped.apply(len) > 1])

area_code
CHN    [Hong Kong, Macao, China, Taiwan]
Name: area, dtype: object


Accordingly, we modify the values in the `area_abbreviation` column.

In [14]:
df_fao.loc[df_fao['area'] == 'Hong Kong', 'area_code'] = 'HKG'
df_fao.loc[df_fao['area'] == 'Macao', 'area_code'] = 'MAC'
df_fao.loc[df_fao['area'] == 'Taiwan', 'area_code'] = 'TWN'

Now, we reshape the DataFrame `df_fao` using the melt function to unpivot it, convert the `year` column to datetime format, and finally, set it as the index of the DataFrame.

In [15]:
# reshape the DataFrame using the melt function to unpivot it
df_fao = pd.melt(df_fao, id_vars=df_fao.columns[:6].tolist(), value_vars=df_fao.columns[6:].astype(str).tolist(),
                 var_name='year', value_name='production')

# specify the date format explicitly
df_fao['year'] = pd.to_datetime(df_fao['year'], format='%Y').dt.year

# set the 'year' column as the index of the DataFrame
df_fao.set_index('year', inplace=True)

As a final step, let's add a column indicating the continent of each country.

In [16]:
converter = coco.CountryConverter()

# creating a list for unique values of countries ISO3 and a list for the continent associated to each country 
area_code_lst = df_fao['area_code'].unique()
continent = converter.convert(names = area_code_lst, src='ISO3', to='continent')

# joining those list in a dictionary
dic = {key:value for key, value in zip(area_code_lst, continent)}

# creating a column in the dataframe 
df_fao['continent'] = df_fao['area_code'].apply(lambda x: dic.get(x))

column_order = ['area_code', 'area', 'continent'] + [col for col in df_fao.columns if col not in ['area_code', 'area', 'continent']]
df_fao = df_fao[column_order]

After the data cleaning and data transformation, the state of the DataFrame `df_fao` is as follows:

In [17]:
df_fao.info()
df_fao.head()

<class 'pandas.core.frame.DataFrame'>
Index: 773959 entries, 1961 to 2013
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   area_code   773959 non-null  object 
 1   area        773959 non-null  object 
 2   continent   773959 non-null  object 
 3   item        773959 non-null  object 
 4   element     773959 non-null  object 
 5   latitude    773959 non-null  float64
 6   longitude   773959 non-null  float64
 7   production  773959 non-null  float64
dtypes: float64(3), object(5)
memory usage: 50.2+ MB


,area_code,area,continent,item,element,latitude,longitude,production
year,,,,,,,,
1961,AFG,Afghanistan,Asia,Wheat and products,Food,33.94,67.71,1928.0
1961,AFG,Afghanistan,Asia,Rice (Milled Equivalent),Food,33.94,67.71,183.0
1961,AFG,Afghanistan,Asia,Barley and products,Feed,33.94,67.71,76.0
1961,AFG,Afghanistan,Asia,Barley and products,Food,33.94,67.71,237.0
1961,AFG,Afghanistan,Asia,Maize and products,Feed,33.94,67.71,210.0


### Food Production dataset

In [18]:
# checking what the dataset looks like
df_prod.info()
df_prod.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 23 columns):
 #   Column                                                                   Non-Null Count  Dtype  
---  ------                                                                   --------------  -----  
 0   Food product                                                             43 non-null     object 
 1   Land use change                                                          43 non-null     float64
 2   Animal Feed                                                              43 non-null     float64
 3   Farm                                                                     43 non-null     float64
 4   Processing                                                               43 non-null     float64
 5   Transport                                                                43 non-null     float64
 6   Packging                                                                 43 

,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Total_emissions,Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal),Eutrophying emissions per kilogram (gPO₄eq per kilogram),Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein),Freshwater withdrawals per 1000kcal (liters per 1000kcal),Freshwater withdrawals per 100g protein (liters per 100g protein),Freshwater withdrawals per kilogram (liters per kilogram),Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal),Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein),Land use per 1000kcal (m² per 1000kcal),Land use per kilogram (m² per kilogram),Land use per 100g protein (m² per 100g protein),Scarcity-weighted water use per kilogram (liters per kilogram),Scarcity-weighted water use per 100g protein (liters per 100g protein),Scarcity-weighted water use per 1000kcal (liters per 1000 kilocalories)
0,Wheat & Rye (Bread),0.1,0.0,0.8,0.2,0.1,0.1,0.1,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Maize (Meal),0.3,0.0,0.5,0.1,0.1,0.1,0.0,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0,1.6,4.281357,11.23,8.638462,183.911552,371.076923,482.4,0.945482,1.907692,2.897446,7.6,5.846154,18786.2,14450.92308,7162.104461
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1,4.0,9.514379,35.07,49.394366,609.983722,3166.760563,2248.4,1.207271,6.267606,0.759631,2.8,3.943662,49576.3,69825.77465,13449.891480


Observing the dataset, we begin to perform the initial cleaning.

In [19]:
# rename columns
df_prod.rename(columns={'Land use per kilogram (m² per kilogram)': 'Land Use'}, inplace=True)
df_prod.rename(columns={'Packging': 'Packaging'}, inplace=True)

# drop columns with parentheses in their names as they are not relevant for analysis
df_prod.drop(columns=[i for i in df_prod.columns if '(' in i], inplace=True)

# rename columns by replacing spaces with underscores and converting to lowercase
df_prod.rename(columns={i: i.replace(' ', '_').lower() for i in df_prod.columns}, inplace=True)

Let's determine the number of NaN values for each column.

In [20]:
# check NaN values
for column in df_prod.columns:
    num_nan = df_prod[column].isna().sum()
    percentage_nan = (num_nan / len(df_prod)) * 100
    print(f"Column '{column}': {percentage_nan:.2f}% NaN")

Column 'food_product': 0.00% NaN
Column 'land_use_change': 0.00% NaN
Column 'animal_feed': 0.00% NaN
Column 'farm': 0.00% NaN
Column 'processing': 0.00% NaN
Column 'transport': 0.00% NaN
Column 'packaging': 0.00% NaN
Column 'retail': 0.00% NaN
Column 'total_emissions': 0.00% NaN
Column 'land_use': 11.63% NaN


For which concerns the NaN values in the `land use` column, we replace them with the median value of the existing values, allowing us to retain the foods with NaN values in this column. This decision enables us to analyze the dataset, preserving emissions data while acknowledging the loss of some data regarding land use.

In [21]:
median_land_use=df_prod['land_use'].median()
df_prod['land_use'] = df_prod['land_use'].fillna(median_land_use)

We can proceed with a similarity test.

In [22]:
check_similarity(df_prod.food_product.unique()).head(3)

,Wheat & Rye (Bread),Maize (Meal),Barley (Beer),Oatmeal,Rice,Potatoes,Cassava,Cane Sugar,Beet Sugar,Other Pulses,Peas,Nuts,Groundnuts,Soymilk,Tofu,Soybean Oil,Palm Oil,Sunflower Oil,Rapeseed Oil,Olive Oil,Tomatoes,Onions & Leeks,Root Vegetables,Brassicas,Other Vegetables,Citrus Fruit,Bananas,Apples,Berries & Grapes,Wine,Other Fruit,Coffee,Dark Chocolate,Beef (beef herd),Beef (dairy herd),Lamb & Mutton,Pig Meat,Poultry Meat,Milk,Cheese,Eggs,Fish (farmed),Shrimps (farmed)
0,"(Wheat & Rye (Bread), 1.0)","(Maize (Meal), 1.0)","(Barley (Beer), 1.0)","(Oatmeal, 1.0)","(Rice, 1.0)","(Potatoes, 1.0)","(Cassava, 1.0)","(Cane Sugar, 1.0)","(Beet Sugar, 1.0)","(Other Pulses, 1.0)","(Peas, 1.0)","(Nuts, 1.0)","(Groundnuts, 1.0)","(Soymilk, 1.0)","(Tofu, 1.0)","(Soybean Oil, 1.0)","(Palm Oil, 1.0)","(Sunflower Oil, 1.0)","(Rapeseed Oil, 1.0)","(Olive Oil, 1.0)","(Tomatoes, 1.0)","(Onions & Leeks, 1.0)","(Root Vegetables, 1.0)","(Brassicas, 1.0)","(Other Vegetables, 1.0)","(Citrus Fruit, 1.0)","(Bananas, 1.0)","(Apples, 1.0)","(Berries & Grapes, 1.0)","(Wine, 1.0)","(Other Fruit, 1.0)","(Coffee, 1.0)","(Dark Chocolate, 1.0)","(Beef (beef herd), 1.0)","(Beef (dairy herd), 1.0)","(Lamb & Mutton, 1.0)","(Pig Meat, 1.0)","(Poultry Meat, 1.0)","(Milk, 1.0)","(Cheese, 1.0)","(Eggs, 1.0)","(Fish (farmed), 1.0)","(Shrimps (farmed), 1.0)"
1,"(Maize (Meal), 0.45)","(Pig Meat, 0.5)","(Maize (Meal), 0.48)","(Maize (Meal), 0.42)","(Wine, 0.5)","(Tomatoes, 0.75)","(Brassicas, 0.5)","(Beet Sugar, 0.7)","(Cane Sugar, 0.7)","(Other Vegetables, 0.64)","(Potatoes, 0.5)","(Groundnuts, 0.43)","(Nuts, 0.43)","(Soybean Oil, 0.56)","(Coffee, 0.4)","(Sunflower Oil, 0.58)","(Olive Oil, 0.59)","(Olive Oil, 0.55)","(Palm Oil, 0.5)","(Palm Oil, 0.59)","(Potatoes, 0.75)","(Berries & Grapes, 0.47)","(Other Vegetables, 0.77)","(Cassava, 0.5)","(Root Vegetables, 0.77)","(Other Fruit, 0.7)","(Brassicas, 0.38)","(Peas, 0.4)","(Onions & Leeks, 0.47)","(Rice, 0.5)","(Citrus Fruit, 0.7)","(Cheese, 0.5)","(Potatoes, 0.36)","(Beef (dairy herd), 0.73)","(Beef (beef herd), 0.73)","(Potatoes, 0.29)","(Poultry Meat, 0.6)","(Pig Meat, 0.6)","(Soymilk, 0.55)","(Coffee, 0.5)","(Peas, 0.25)","(Shrimps (farmed), 0.76)","(Fish (farmed), 0.69)"
2,"(Barley (Beer), 0.44)","(Barley (Beer), 0.48)","(Wheat & Rye (Bread), 0.44)","(Other Pulses, 0.42)","(Pig Meat, 0.33)","(Peas, 0.5)","(Peas, 0.36)","(Maize (Meal), 0.36)","(Beef (dairy herd), 0.44)","(Other Fruit, 0.61)","(Pig Meat, 0.5)","(Potatoes, 0.33)","(Citrus Fruit, 0.36)","(Milk, 0.55)","(Tomatoes, 0.33)","(Soymilk, 0.56)","(Soybean Oil, 0.53)","(Soybean Oil, 0.5)","(Olive Oil, 0.48)","(Sunflower Oil, 0.55)","(Oatmeal, 0.4)","(Other Pulses, 0.31)","(Potatoes, 0.43)","(Bananas, 0.38)","(Other Pulses, 0.64)","(Other Pulses, 0.33)","(Peas, 0.36)","(Other Pulses, 0.33)","(Shrimps (farmed), 0.38)","(Onions & Leeks, 0.33)","(Other Pulses, 0.61)","(Tofu, 0.4)","(Tomatoes, 0.36)","(Wheat & Rye (Bread), 0.4)","(Fish (farmed), 0.47)","(Palm Oil, 0.29)","(Maize (Meal), 0.5)","(Potatoes, 0.5)","(Maize (Meal), 0.38)","(Other Pulses, 0.44)","(Nuts, 0.25)","(Beef (dairy herd), 0.47)","(Beef (dairy herd), 0.42)"


After the data cleaning and data transformation, the state of the DataFrame `df_prod` is as follows:

In [23]:
df_prod.info()
df_prod.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   food_product     43 non-null     object 
 1   land_use_change  43 non-null     float64
 2   animal_feed      43 non-null     float64
 3   farm             43 non-null     float64
 4   processing       43 non-null     float64
 5   transport        43 non-null     float64
 6   packaging        43 non-null     float64
 7   retail           43 non-null     float64
 8   total_emissions  43 non-null     float64
 9   land_use         43 non-null     float64
dtypes: float64(9), object(1)
memory usage: 3.5+ KB


,food_product,land_use_change,animal_feed,farm,processing,transport,packaging,retail,total_emissions,land_use
0,Wheat & Rye (Bread),0.1,0.0,0.8,0.2,0.1,0.1,0.1,1.4,6.865
1,Maize (Meal),0.3,0.0,0.5,0.1,0.1,0.1,0.0,1.1,6.865
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3,1.1,6.865
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0,1.6,7.600
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1,4.0,2.800
5,Potatoes,0.0,0.0,0.2,0.0,0.1,0.0,0.0,0.3,0.880
6,Cassava,0.6,0.0,0.2,0.0,0.1,0.0,0.0,0.9,1.810
7,Cane Sugar,1.2,0.0,0.5,0.0,0.8,0.1,0.0,2.6,2.040
8,Beet Sugar,0.0,0.0,0.5,0.2,0.6,0.1,0.0,1.4,1.830
9,Other Pulses,0.0,0.0,1.1,0.0,0.1,0.4,0.0,1.6,15.570


### Population dataset

In [24]:
# checking what the dataset looks like
df_pop.info()
df_pop.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 68 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    266 non-null    object 
 1   Country Code    266 non-null    object 
 2   Indicator Name  266 non-null    object 
 3   Indicator Code  266 non-null    object 
 4   1960            264 non-null    float64
 5   1961            264 non-null    float64
 6   1962            264 non-null    float64
 7   1963            264 non-null    float64
 8   1964            264 non-null    float64
 9   1965            264 non-null    float64
 10  1966            264 non-null    float64
 11  1967            264 non-null    float64
 12  1968            264 non-null    float64
 13  1969            264 non-null    float64
 14  1970            264 non-null    float64
 15  1971            264 non-null    float64
 16  1972            264 non-null    float64
 17  1973            264 non-null    flo

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54608.0,55811.0,56682.0,57475.0,58178.0,58782.0,59291.0,59522.0,59471.0,59330.0,59106.0,58816.0,58855.0,59365.0,60028.0,60715.0,61193.0,61465.0,61738.0,62006.0,62267.0,62614.0,63116.0,63683.0,64174.0,64478.0,64553.0,64450.0,64332.0,64596.0,65712.0,67864.0,70192.0,72360.0,74710.0,77050.0,79417.0,81858.0,84355.0,86867.0,89101.0,90691.0,91781.0,92701.0,93540.0,94483.0,95606.0,96787.0,97996.0,99212.0,100341.0,101288.0,102112.0,102880.0,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0,106537.0,106445.0,NaN
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130692579.0,134169237.0,137835590.0,141630546.0,145605995.0,149742351.0,153955516.0,158313235.0,162875171.0,167596160.0,172475766.0,177503186.0,182599092.0,187901657.0,193512956.0,199284304.0,205202669.0,211120911.0,217481420.0,224315978.0,230967858.0,237937461.0,245386717.0,252779730.0,260209149.0,267938123.0,276035920.0,284490394.0,292795186.0,301124880.0,309890664.0,318544083.0,326933522.0,335625136.0,344418362.0,353466601.0,362985802.0,372352230.0,381715600.0,391486231.0,401600588.0,412001885.0,422741118.0,433807484.0,445281555.0,457153837.0,469508516.0,482406426.0,495748900.0,509410477.0,523459657.0,537792950.0,552530654.0,567892149.0,583651101.0,600008424.0,616377605.0,632746570.0,649757148.0,667242986.0,685112979.0,702977106.0,720859132.0,NaN
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8622466.0,8790140.0,8969047.0,9157465.0,9355514.0,9565147.0,9783147.0,10010030.0,10247780.0,10494489.0,10752971.0,11015857.0,11286753.0,11575305.0,11869879.0,12157386.0,12425267.0,12687301.0,12938862.0,12986369.0,12486631.0,11155195.0,10088289.0,9951449.0,10243686.0,10512221.0,10448442.0,10322758.0,10383460.0,10673168.0,10694796.0,10745167.0,12057433.0,14003760.0,15455555.0,16418912.0,17106595.0,17788819.0,18493132.0,19262847.0,19542982.0,19688632.0,21000256.0,22645130.0,23553551.0,24411191.0,25442944.0,25903301.0,26427199.0,27385307.0,28189672.0,29249157.0,30466479.0,31541209.0,32716210.0,33753499.0,34636207.0,35643418.0,36686784.0,37769499.0,38972230.0,40099462.0,41128771.0,NaN
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,97256290.0,99314028.0,101445032.0,103667517.0,105959979.0,108336203.0,110798486.0,113319950.0,115921723.0,118615741.0,121424797.0,124336039.0,127364044.0,130563107.0,133953892.0,137548613.0,141258400.0,145122851.0,149206663.0,153459665.0,157825609.0,162323313.0,167023385.0,171566640.0,176054495.0,180817312.0,185720244.0,190759952.0,195969722.0,201392200.0,206739024.0,212172888.0,217966101.0,223788766.0,229675775.0,235861484.0,242200260.0,248713095.0,255482918.0,262397030.0,269611898.0,277160097.0,284952322.0,292977949.0,301265247.0,309824829.0,318601484.0,327612838.0,336893835.0,346475221.0,356337762.0,366489204.0,376797999.0,387204553.0,397855507.0,408690375.0,419778384.0,431138704.0,442646825.0,454306063.0,466189102.0,478185907.0,490330870.0,NaN
4,Angola,AGO,"Population, total",SP.POP.TOTL,5357195.0,5441333.0,5521400.0,5599827.0,5673199.0,5736582.0,5787044.0,5827503.0,5868203.0,5928386.0,6029700.0,6177049.0,6364731.0,6578230.0,6802494.0,7032713.0,7266780.0,7511895.0,7771590.0,8043218.0,8330047.0,8631457.0,8947152.0,9276707.0,9617702.0,9970621.0,10332574.0,10694057.0,11060261.0,11439498.0,11828638.0,12228691.0,12632507.0,13038270.0,13462031.0,13912253.0,14383350.0,14871146.0,15366864.0,15870753.0,16394062.0,16941587.0,17516139.0,18124342.0,18771125.0,19450959.0,20162340.0,20909684.0,21691522.0,22507674.0,23364185.0,24259111.0,25188292.0,26147002.0,27128337.0,28127721.0,29154746.0,30208628.0,312735

Observing the dataset, we begin to perform the initial cleaning.

In [25]:
# drop columns after 2013
df_pop=df_pop.iloc[:,:58]

# drop columns from the DataFrame
df_pop.drop(columns=['Indicator Name','Indicator Code', '1960'], inplace=True)

# rename columns by replacing spaces with underscores and converting to lowercase
df_pop.rename(columns={i: i.replace(' ', '_').lower() for i in df_pop.columns}, inplace=True)

Let's keep only the data on the population of the countries considered in the analysis.

In [26]:
# get the unique values of area_code from df_fao
area_codes = df_fao['area_code'].unique()

# keep only the rows of df_pop that have a country_code present in area_code
df_pop = df_pop[df_pop['country_code'].isin(area_codes)]

In [27]:
# get the unique values of country_code from df_pop_filtered
unique_country_codes = df_pop['country_code'].unique()

# find the missing values by comparing the two sets
missing_country_codes = set(area_codes) - set(unique_country_codes)

# print the missing values
if missing_country_codes:
    print("The following country_code are present in area_code of df_fao but are missing in df_pop:")
    for code in missing_country_codes:
        print(code)
else:
    print("There are no missing country_code.")

The following country_code are present in area_code of df_fao but are missing in df_pop:
TWN


Taiwan is not present in `df_pop`. According to the documentation, Taiwan is included in the population of China. To maintain consistency with the previous DataFrames, let's add the population of Taiwan and subtract it from China.

In [28]:
df_pop = pd.concat([df_pop, df_pop_taiwan], ignore_index=True)

# subtract the values of TWN from those of CHN for each year
df_pop.loc[df_pop['country_name'] == 'China', '1961':'2013'] -= df_pop.loc[df_pop['country_name'] == 'Taiwan', '1961':'2013'].values

# sort df_pop alphabetically based on the 'country_name' column
df_pop = df_pop.sort_values(by='country_name')

As a final step, let's add a column indicating the continent of each country.

In [29]:
converter = coco.CountryConverter()

# creating a list for unique values of countries ISO3 and a list for the continent associated to each country 
country_code_lst = df_pop['country_code'].unique()
continent = converter.convert(names = country_code_lst, src='ISO3', to='continent')

# joining those list in a dictionary
dic = {key:value for key, value in zip(country_code_lst, continent)}

# creating a column in the dataframe 
df_pop['continent'] = df_pop['country_code'].apply(lambda x: dic.get(x))

column_order = ['country_name', 'country_code', 'continent'] + [col for col in df_pop.columns if col not in ['country_name', 'country_code', 'continent']]
df_pop = df_pop[column_order]

After the data cleaning and data transformation, the state of the DataFrame `df_pop` is as follows:

In [30]:
# reset index
df_pop.reset_index(drop=True, inplace=True)
df_pop

,country_name,country_code,continent,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Afghanistan,AFG,Asia,8790140.0,8969047.0,9157465.0,9355514.0,9565147.0,9783147.0,10010030.0,10247780.0,10494489.0,10752971.0,11015857.0,11286753.0,11575305.0,11869879.0,12157386.0,12425267.0,12687301.0,12938862.0,12986369.0,12486631.0,11155195.0,10088289.0,9951449.0,10243686.0,10512221.0,10448442.0,10322758.0,10383460.0,10673168.0,10694796.0,10745167.0,12057433.0,14003760.0,15455555.0,16418912.0,17106595.0,17788819.0,18493132.0,19262847.0,19542982.0,19688632.0,21000256.0,22645130.0,23553551.0,24411191.0,25442944.0,25903301.0,26427199.0,27385307.0,28189672.0,29249157.0,30466479.0,31541209.0
1,Albania,ALB,Europe,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,2081695.0,2135479.0,2187853.0,2243126.0,2296752.0,2350124.0,2404831.0,2458526.0,2513546.0,2566266.0,2617832.0,2671997.0,2726056.0,2784278.0,2843960.0,2904429.0,2964762.0,3022635.0,3083605.0,3142336.0,3227943.0,3286542.0,3266790.0,3247039.0,3227287.0,3207536.0,3187784.0,3168033.0,3148281.0,3128530.0,3108778.0,3089027.0,3060173.0,3051010.0,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0
2,Algeria,DZA,Africa,11598608.0,11778260.0,11969451.0,12179099.0,12381256.0,12613389.0,12897115.0,13190975.0,13491016.0,13795915.0,14110271.0,14439748.0,14786469.0,15153602.0,15724692.0,16500516.0,17134192.0,17632645.0,18166981.0,18739378.0,19351357.0,20000096.0,20682111.0,21393530.0,22132905.0,22882553.0,23586101.0,24243018.0,24889507.0,25518074.0,26133905.0,26748303.0,27354327.0,27937006.0,28478022.0,28984634.0,29476031.0,29924668.0,30346083.0,30774621.0,31200985.0,31624696.0,32055883.0,32510186.0,32956690.0,33435080.0,33983827.0,34569592.0,35196037.0,35856344.0,36543541.0,37260563.0,38000626.0
3,Angola,AGO,Africa,5441333.0,5521400.0,5599827.0,5673199.0,5736582.0,5787044.0,5827503.0,5868203.0,5928386.0,6029700.0,6177049.0,6364731.0,6578230.0,6802494.0,7032713.0,7266780.0,7511895.0,7771590.0,8043218.0,8330047.0,8631457.0,8947152.0,9276707.0,9617702.0,9970621.0,10332574.0,10694057.0,11060261.0,11439498.0,11828638.0,12228691.0,12632507.0,13038270.0,13462031.0,13912253.0,14383350.0,14871146.0,15366864.0,15870753.0,16394062.0,16941587.0,17516139.0,18124342.0,18771125.0,19450959.0,20162340.0,20909684.0,21691522.0,22507674.0,23364185.0,24259111.0,25188292.0,26147002.0
4,Antigua and Barbuda,ATG,America,56245.0,57008.0,57778.0,58664.0,59644.0,60615.0,61617.0,62658.0,63742.0,64517.0,64769.0,64695.0,64463.0,64218.0,64035.0,63990.0,64117.0,64329.0,64609.0,64889.0,65076.0,65231.0,65408.0,65362.0,65048.0,64712.0,64353.0,63982.0,63636.0,63328.0,63634.0,64659.0,65834.0,67072.0,68398.0,69798.0,71218.0,72572.0,73821.0,75055.0,76215.0,77195.0,78075.0,78941.0,79869.0,80895.0,82016.0,83251.0,84534.0,85695.0,86729.0,87674.0,88497.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,"Venezuela, RB",VEN,America,8453106.0,8754082.0,9059953.0,9371333.0,9688138.0,10010685.0,10338848.0,10672654.0,11011335.0,11355475.0,11705674.0,12061621.0,12424804.0,12796220.0,13176387.0,13566112.0,13965250.0,14372959.0,14788523.0,15210443.0,15638426.0,16071654.0,16509596.0,16953234.0,17402304.0,17860872.0,18328655.0,18800770.0,19275292.0,19750579.0,20226214.0,20700461.0,21172100.0,21640833.0,22107286.0,22572110.0,23037561.0,23503819.0,23966960.0,24427729.0,24880203.0,25330929.0,25782029.0,26226927.0,26668785.0,27102081.0,27525097.0,27933833.0,28327892.0,28715022.0,29096159.0,29470426.0,29838021.0
142,Viet Nam,VNM,Asia,33621982.0,34533889.0,35526727.0,36509166.0,3746

### Temperature dataset

In [31]:
# checking what the dataset looks like
df_temp.info()
df_temp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9656 entries, 0 to 9655
Data columns (total 66 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Area Code     9656 non-null   int64  
 1   Area          9656 non-null   object 
 2   Months Code   9656 non-null   int64  
 3   Months        9656 non-null   object 
 4   Element Code  9656 non-null   int64  
 5   Element       9656 non-null   object 
 6   Unit          9656 non-null   object 
 7   Y1961         8287 non-null   float64
 8   Y1962         8322 non-null   float64
 9   Y1963         8294 non-null   float64
 10  Y1964         8252 non-null   float64
 11  Y1965         8281 non-null   float64
 12  Y1966         8364 non-null   float64
 13  Y1967         8347 non-null   float64
 14  Y1968         8345 non-null   float64
 15  Y1969         8326 non-null   float64
 16  Y1970         8308 non-null   float64
 17  Y1971         8303 non-null   float64
 18  Y1972         8323 non-null 

,Area Code,Area,Months Code,Months,Element Code,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,Afghanistan,7001,January,7271,Temperature change,°C,0.777,0.062,2.744,-5.232,1.868,3.629,-1.432,0.389,-2.298,0.804,-1.487,-1.305,-2.951,-1.184,-0.490,2.409,-3.014,-0.663,1.141,-0.393,1.724,0.678,0.524,-0.058,0.435,0.332,2.655,1.150,-1.108,0.634,0.018,0.582,-0.821,1.087,1.297,-0.718,1.426,0.950,0.859,1.565,-0.603,1.606,2.479,2.707,0.109,-1.606,0.431,-5.553,1.518,3.601,1.179,-0.583,1.233,1.755,1.943,3.416,1.201,1.996,2.951
1,2,Afghanistan,7001,January,6078,Standard Deviation,°C,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950
2,2,Afghanistan,7002,February,7271,Temperature change,°C,-1.743,2.465,3.919,-0.202,-0.096,3.397,0.296,-2.055,-3.167,1.809,0.816,-7.722,1.838,-3.706,-1.239,-1.620,-0.156,-0.369,1.072,-1.222,1.088,-2.101,0.460,-4.321,2.467,-0.286,1.409,0.170,-2.890,-0.310,-1.373,-0.120,1.414,-1.412,-0.149,0.870,0.043,-0.540,3.222,-0.901,0.707,0.985,1.816,2.871,-1.506,4.725,1.645,-2.332,2.494,1.212,0.321,-3.201,1.494,-3.187,2.699,2.251,-0.323,2.705,0.086
3,2,Afghanistan,7002,February,6078,Standard Deviation,°C,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597
4,2,Afghanistan,7003,March,7271,Temperature change,°C,0.516,1.336,0.403,1.659,-0.909,-0.069,-0.759,0.496,2.481,-0.915,1.658,-1.784,-0.473,1.001,-0.585,-2.817,3.377,-1.536,-1.420,-0.628,1.166,-1.781,-2.406,1.761,-0.160,-3.087,1.330,-0.056,0.065,-1.000,-0.901,-2.220,-1.449,1.312,-1.451,-0.336,-0.005,-0.576,-0.217,-0.267,1.229,1.949,-0.158,2.753,1.663,1.847,-0.201,4.172,2.362,3.390,0.748,-0.527,2.246,-0.076,-0.497,2.296,0.834,4.418,0.234


Observing the dataset, we begin to perform the initial cleaning.

In [32]:
# select continents
df_temp=df_temp[df_temp['Area'].isin(['Africa', 'Americas', 'Asia', 'Europe', 'Oceania','World'])]

# select temperature change
df_temp = df_temp.loc[df_temp.Element == 'Temperature change']

# drop columns after 2013
df_temp=df_temp.iloc[:,:60]

# drop columns from the DataFrame
df_temp.drop(columns=['Area Code', 'Months Code', 'Element Code', 'Element', 'Unit'], inplace=True)

# rename columns containing 'Y' by removing the 'Y' character
df_temp.rename(columns={x: x[1:] for x in df_temp.columns if 'Y' in x}, inplace=True)

# rename columns
df_temp.rename(columns={'Area': 'country_name', 'Months':'months'}, inplace=True)

# fix decryption errors
df_temp['months'] = df_temp['months'].replace({
    'Mar\x96Apr\x96May': 'Spring',
    'Jun\x96Jul\x96Aug': 'Summer',
    'Sep\x96Oct\x96Nov': 'Fall',
    'Dec\x96Jan\x96Feb': 'Winter'
})

Let's determine the number of NaN values for each column.

In [33]:
# check NaN values
for column in df_temp.columns:
    num_nan = df_temp[column].isna().sum()
    percentage_nan = (num_nan / len(df_temp)) * 100
    print(f"Column '{column}': {percentage_nan:.2f}% NaN")

Column 'country_name': 0.00% NaN
Column 'months': 0.00% NaN
Column '1961': 0.00% NaN
Column '1962': 0.00% NaN
Column '1963': 0.00% NaN
Column '1964': 0.00% NaN
Column '1965': 0.00% NaN
Column '1966': 0.00% NaN
Column '1967': 0.00% NaN
Column '1968': 0.00% NaN
Column '1969': 0.00% NaN
Column '1970': 0.00% NaN
Column '1971': 0.00% NaN
Column '1972': 0.00% NaN
Column '1973': 0.00% NaN
Column '1974': 0.00% NaN
Column '1975': 0.00% NaN
Column '1976': 0.00% NaN
Column '1977': 0.00% NaN
Column '1978': 0.00% NaN
Column '1979': 0.00% NaN
Column '1980': 0.00% NaN
Column '1981': 0.00% NaN
Column '1982': 0.00% NaN
Column '1983': 0.00% NaN
Column '1984': 0.00% NaN
Column '1985': 0.00% NaN
Column '1986': 0.00% NaN
Column '1987': 0.00% NaN
Column '1988': 0.00% NaN
Column '1989': 0.00% NaN
Column '1990': 0.00% NaN
Column '1991': 0.00% NaN
Column '1992': 0.00% NaN
Column '1993': 0.00% NaN
Column '1994': 0.00% NaN
Column '1995': 0.00% NaN
Column '1996': 0.00% NaN
Column '1997': 0.00% NaN
Column '1998': 

Now, we reshape the DataFrame `df_temp` using the melt function to unpivot it, convert the `year` column to datetime format, and finally, set it as the index of the DataFrame.

In [34]:
# reshape the DataFrame using the melt function to unpivot it
df_temp = pd.melt(df_temp, id_vars=df_temp.columns[:2].tolist(), value_vars=df_temp.columns[2:].astype(str).tolist(),
                 var_name='year', value_name='temp')

# specify the date format explicitly
df_temp['year'] = pd.to_datetime(df_temp['year'], format='%Y').dt.year

# set the 'year' column as the index of the DataFrame
df_temp.set_index('year', inplace=True)

After the data cleaning and data transformation, the state of the DataFrame `df_fao` is as follows:

In [35]:
df_temp

,country_name,months,temp
year,,,
1961,World,January,0.399
1961,World,February,0.263
1961,World,March,0.254
1961,World,April,0.278
1961,World,May,0.367
...,...,...,...
2013,Oceania,Winter,1.397
2013,Oceania,Spring,1.146
2013,Oceania,Summer,1.385


# Data Exploration & Data Visualization

We can proceed with exploring and visualizing the data. First, let's observe the trends of food and feed production and population growth from 1961 to 2013.

In [36]:
# group by year and element, then sum up the production
production_data = df_fao.groupby(['year', 'element'])['production'].sum().reset_index()

# min and max years
min_year = production_data['year'].min()
max_year = production_data['year'].max()

# first and last food and feed productions
first_data = production_data[production_data['year'] == min_year]
last_data = production_data[production_data['year'] == max_year]
first_food_production = first_data[first_data['element'] == 'Food']['production'].values[0]
first_feed_production = first_data[first_data['element'] == 'Feed']['production'].values[0]
last_food_production = last_data[last_data['element'] == 'Food']['production'].values[0]
last_feed_production = last_data[last_data['element'] == 'Feed']['production'].values[0]

# percentage differences
difference_percentage_first = ((first_food_production - first_feed_production) / first_feed_production) * 100
difference_percentage_last = ((last_food_production - last_feed_production) / last_feed_production) * 100

# create traces for food and feed productions
food_trace = go.Scatter(x=production_data[production_data['element'] == 'Food']['year'],
                        y=production_data[production_data['element'] == 'Food']['production'],
                        mode='lines',
                        name='Food')

feed_trace = go.Scatter(x=production_data[production_data['element'] == 'Feed']['year'],
                        y=production_data[production_data['element'] == 'Feed']['production'],
                        mode='lines',
                        name='Feed')

# create vertical lines for the first and last points
first_line = go.Scatter(x=[min_year, min_year],
                        y=[first_food_production, first_feed_production],
                        mode='lines',
                        line=dict(color='green', dash='dash'),
                        name=f'percentage difference: {min_year}')

last_line = go.Scatter(x=[max_year, max_year],
                       y=[last_food_production, last_feed_production],
                       mode='lines',
                       line=dict(color='green', dash='dash'),
                       name=f'percentage difference: {max_year}')

# add annotations for percentage differences
annotations = [
    dict(x=min_year, y=(first_food_production + first_feed_production) / 2,
         text=f'{difference_percentage_first:.2f}%',
         showarrow=True,
         arrowhead=4,
         ax=50,
         ay=-50),
    dict(x=max_year, y=(last_food_production + last_feed_production) / 2,
         text=f'{difference_percentage_last:.2f}%',
         showarrow=True,
         arrowhead=4,
         ax=-50,
         ay=-50)
]

# create the figure
fig = go.Figure(data=[food_trace, feed_trace, first_line, last_line],
                layout=dict(title=dict(text=f'Food and feed production from {min_year} to {max_year}',
                                        x=0.5),  # center title
                            xaxis=dict(title='Year', tickmode='linear', tick0=min_year, dtick=5,
                                       range=[min_year - 3, max_year + 3]),  # set axis range
                            yaxis=dict(title='Production (thousands of tons)'),
                            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                            annotations=annotations,
                            width=1800,  # set width
                            height=800))  # set height

# show the plot
fig.show()

# select only the columns of interest (from 1961 onwards) and calculate the sum for each year
pop_tot_years = df_pop.iloc[:, 3:].sum()

# percentage difference
difference_percentage_pop = ((pop_tot_years.iloc[-1] - pop_tot_years.iloc[1]) / pop_tot_years.iloc[1]) * 100

# create trace for population
pop_trace = go.Scatter(x=production_data[production_data['element'] == 'Food']['year'],
                        y=pop_tot_years,
                        mode='lines',
                        name='Population')

# create the figure
fig = go.Figure(data=pop_trace,
                layout=dict(title=dict(text=f'Population from {min_year} to {max_year}, '
                                             f'Population Growth: {difference_percentage_pop:.2f}%',
                                        x=0.5),  # center title
                            xaxis=dict(title='Year', tickmode='linear', tick0=min_year, dtick=5,
                                       range=[min_year - 3, max_year + 3]),  # set axis range
                            yaxis=dict(title='Population'),
                            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                            width=1800,  # set width
                            height=800))  # set height

# show the plot
fig.show()

# production and population data for the first and last year
first_year_data = production_data.loc[production_data['year'] == min_year]
last_year_data = production_data.loc[production_data['year'] == max_year]
first_pop = pop_tot_years.iloc[0]
last_pop = pop_tot_years.iloc[-1]

# calculate percentage difference
production_change_percentage = ((last_year_data['production'].sum() - first_year_data['production'].sum()) / 
                                first_year_data['production'].sum()) * 100

# calculate percentage positions for the dashed line
y_position_prod = [first_year_data['production'].sum(), last_year_data['production'].sum()]
y_position_pop = [first_pop, last_pop]

# create figure with two subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Total Production", "Population"))

# data for the first subplot (Production)
fig.add_trace(go.Bar(x=[0, 1], 
                     y=[first_year_data['production'].sum(), last_year_data['production'].sum()],
                     marker=dict(color=['orange', 'yellow']),
                     name="Production"),
              row=1, col=1)

# data for the second subplot (Population)
fig.add_trace(go.Bar(x=[0, 1], 
                     y=[first_pop, last_pop],
                     marker=dict(color=['orange', 'yellow']),
                     name="Population"),
              row=1, col=2)

# set custom labels on the x-axis
fig.update_xaxes(tickvals=[0, 1], ticktext=[min_year, max_year], row=1, col=1)
fig.update_xaxes(tickvals=[0, 1], ticktext=[min_year, max_year], row=1, col=2)

# set titles and axis labels for the Production subplot
fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_yaxes(title_text="Production (thousands of tons)", row=1, col=1)

# set titles and axis labels for the Population subplot
fig.update_xaxes(title_text="Year", row=1, col=2)
fig.update_yaxes(title_text="Population", row=1, col=2)

# add dashed line annotation for Production increase
fig.add_shape(type="line",
              x0=0.5, y0=y_position_prod[0],
              x1=0.5, y1=y_position_prod[1],
              line=dict(color="red", width=2, dash="dash"),
              row=1, col=1)

# add dashed line annotation for Population increase
fig.add_shape(type="line",
              x0=0.5, y0=y_position_pop[0],
              x1=0.5, y1=y_position_pop[1],
              line=dict(color="red", width=2, dash="dash"),
              row=1, col=2)

# add annotation for the percentage increase in Production
fig.add_annotation(x=0.4, y=y_position_prod[0] + (y_position_prod[1] - y_position_prod[0]) / 2,
                   text=f"+{production_change_percentage:.2f}%",
                   showarrow=False,
                   font=dict(color="red", size=12),
                   row=1, col=1)

# add annotation for the percentage increase in Population
fig.add_annotation(x=0.4, y=y_position_pop[0] + (y_position_pop[1] - y_position_pop[0]) / 2,
                   text=f"+{difference_percentage_pop:.2f}%",
                   showarrow=False,
                   font=dict(color="red", size=12),
                   row=1, col=2)

# set layout of the figure with specified width and height
fig.update_layout(title_text="Total Production and Population", 
                  title_xanchor="center", title_yanchor="top",
                  title_x=0.5, title_y=0.95,
                  width=1800, height=800) 

# show the figure
fig.show()

Examining the first plot, we can notice a consistently increasing trend, as one might expect due to the global population growth (second plot) and the subsequent need for more food and feed.
Indeed, in 1961, food production exceeded feed production by 194%, whereas in 2013 it exceeded it by 277%.
We can motivate the increase in the production ratio between food and feed over the years to changes in dietary preferences, population growth, urbanization, agricultural technology enhancements, more efficient agricultural practices, economic and political changes influencing food production and distribution.

Given the 277.15% increase in food production and the 137.28% population increase, we can ask ourselves the following question: How much food did a person have available in 1961 and 2013?

Assuming that the calculation is performed considering that the entire population can access the food produced equally, the results are:

- 1961: 0.88 tons
- 2013: 1.27 tons

Looking at these values, the first question is: If each person has more food every year, thanks to various factors, why is there still a considerable portion of the population suffering and dying of hunger? Consequently, other questions arise such as: How much food does each person wastes annually and how are these data linked to food problems? These questions deserve a dedicated notebook and a detailed analysis.

---

Now let's observe the trend of production (both food and feed) over the years from 1961 to 2013, divided by continent through an interactive plot.

In [37]:
# yearly production by continent
prod_by_cont_df = df_fao.groupby(['continent', 'year'])['production'].sum().reset_index()

# define colors for continents
colors = {'Africa': 'black', 'Asia': 'yellow', 'Europe': 'green', 'America': 'red', 'Oceania': 'blue'}


prod_by_cont_df['color']=prod_by_cont_df['continent'].map(colors)

# plot representation
my_raceplot = barplot(prod_by_cont_df,  item_column='continent', value_column='production', time_column='year', top_entries=10, item_color='color')
fig=my_raceplot.plot(item_label = 'Continents', value_label = 'Production (Tons)', frame_duration = 150, date_format='%Y', orientation='horizontal')

# layout
fig.update_layout(
      title='Production by continents (1961-2013)',
      title_x=0.5,
      title_y=0.9,
      title_xanchor='center',     
      title_yanchor='top',
      width=1800,
      height=800)

Looking at the trend of production over the years, the growth of Asia is clearly evident. The table below shows the increase in production for each continent in percentage:

| Continent   | Percentage Increase (1961-2013) |
|----------|---------------------------------|
| Africa   | 440%                         |
| Asia     | 401.3%                         |
| Oceania  | 158%                         |
| America  | 135.2%                         |
| Europe   | 29.4%                          |

Africa and Asia stand out as continents that have experienced significant increases in their food and feed production capacities. The only continent that has not even doubled its production is Europe. These data are influenced by many factors, but certainly the presence of many emerging countries with rapidly growing populations in Asia and Africa plays a major role, as we will see in the subsequent plot.

| Year | 1° Continent | 2° Continent | Percentage Difference |
|------|-------------|-------------|-----------------------|
| 1961 | Asia        | Europe      | 37.3%                |
| 2013 | Asia        | America     | 212.8%              |

Another remarkable statistic representing the incredible development of Asian countries is the difference in production between the first and second continents at the beginning and end of our analysis period.

---

Now let's observe the trend of population over the years from 1961 to 2013, divided by continent through an interactive plot.

In [38]:
# remove unnecessary columns and group by continent
grouped_continent = df_pop.drop(['country_name', 'country_code'], axis=1).groupby('continent').sum().reset_index()

# transform using melt to achieve the format suitable for the animated graph
grouped_continent = pd.melt(grouped_continent, id_vars=['continent'], var_name='year', value_name='population')

# sort by continent and year
grouped_continent = grouped_continent.sort_values(by=['continent', 'year']).reset_index(drop=True)

# define colors for the continents
grouped_continent['color'] = grouped_continent['continent'].map(colors)

# graph representation
my_raceplot = barplot(grouped_continent,  item_column='continent', value_column='population', time_column='year', top_entries=10, item_color='color')
fig=my_raceplot.plot(item_label = 'Continents', value_label = 'Population', frame_duration = 150, date_format='%Y', orientation='horizontal')

# layout
fig.update_layout(
      title='Population by continents (1961-2013)',
      title_x=0.5,
      title_y=0.9,
      title_xanchor='center',     
      title_yanchor='top',
      width=1800,
      height=800)

| Continent | Percentage Increase (1961-2013) |
|-----------|---------------------------------|
| Africa    | 286.6%                         |
| Asia      | 151.7%                         |
| Oceania   | 117.9%                         |
| America   | 125%                         |
| Europe    | 24.2%                          |

Africa experienced the most significant increase, with an impressive growth of 286.64%, followed by Asia, America, and Oceania. In contrast, Europe showed the lowest growth rate, with only a 24.20% increase during the same period. This highlights substantial differences in demographic growth rates among continents over fifty-two years.

| Year | 1° Continent | 2° Continent | Percentage Difference |
|------|-------------|-------------|-----------------------|
| 1961 | Asia        | America     | 292.5%               |
| 2013 | Asia        | America     | 339.1%               |

It's interesting to note the changing percentage difference between Asia and America, the leading and second-ranking continents at the beginning and end of the analysis period, highlighting a widening gap over time.

---

Now let's observe the top 15 food items by production in 1961 and 2013, highlighting the associated country.

In [39]:
# filter the data to include only the year 1961 and the 'Food' elements
df_1961_food = df_fao[(df_fao.index == 1961) & (df_fao['element'] == 'Food')]
df_2013_food = df_fao[(df_fao.index == 2013) & (df_fao['element'] == 'Food')]

# sort the filtered DataFrame based on the 'production' column in descending order
top_15_production_items_1961_food = df_1961_food.groupby(['item', 'area']).sum().sort_values(by='production', ascending=False).head(15).reset_index()
top_15_production_items_2013_food = df_2013_food.groupby(['item', 'area']).sum().sort_values(by='production', ascending=False).head(15).reset_index()

# divide the values on the y-axis by 1000 (thousands)
top_15_production_items_1961_food['production'] /= 1000
top_15_production_items_2013_food['production'] /= 1000

# get a list of unique areas
all_areas = list(set(top_15_production_items_1961_food['area'].unique()).union(set(top_15_production_items_2013_food['area'].unique())))

# define colors using Plotly's color scale
colors = px.colors.qualitative.Plotly

# create figure for 1961
fig_1961 = go.Figure()
for i, area in enumerate(all_areas):
    data_area = top_15_production_items_1961_food[top_15_production_items_1961_food['area'] == area]
    fig_1961.add_trace(go.Bar(x=data_area['item'], y=data_area['production'], name=area, marker_color=colors[i % len(colors)]))

fig_1961.update_layout(
    title=dict(text='Top 15 Food Items by Production with Respective Areas - Year 1961', x=0.5),
    xaxis=dict(title='Item'),
    yaxis=dict(title='Production (thousands of tons)', ticksuffix='k'),  # add 'k' suffix to tick labels
    barmode='stack',
    width=1800,  # set width to 1800 pixels
    height=400,   # set height to 400 pixels
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

# create figure for 2013
fig_2013 = go.Figure()
for i, area in enumerate(all_areas):
    data_area = top_15_production_items_2013_food[top_15_production_items_2013_food['area'] == area]
    fig_2013.add_trace(go.Bar(x=data_area['item'], y=data_area['production'], name=area, marker_color=colors[i % len(colors)]))

fig_2013.update_layout(
    title=dict(text='Top 15 Food Items by Production with Respective Areas - Year 2013', x=0.5),
    xaxis=dict(title='Item'),
    yaxis=dict(title='Production (thousands of tons)', ticksuffix='k'),  # add 'k' suffix to tick labels
    barmode='stack',
    width=1800,  # set width to 1800 pixels
    height=400,   # set height to 400 pixels
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

# Show the plots
fig_1961.show()
fig_2013.show()

Several considerations can be made regarding food production along the two years considered:

- **Changes in food production between the two years**: There is a significant increase in food production, especially in China and India, between 1961 and 2013. This can be due to a combination of factors, including technological development, more widespread use of fertilizers and pesticides, as well as improvements in agricultural practices.

- **Diversification of production**: In 2013, there is a greater diversification of food production compared to 1961. While in 1961 starchy roots and sweet potatoes dominated production in China, by 2013, vegetables, cereals, and fruits had massively increased, reflecting greater dietary variety and possible changes in food preferences.

- **Increase in cereal production**: There is a significant increase in cereal production, both in China and India, in 2013 compared to 1961. This could be due to the growing demand of cereals as a primary source for animal feed.

- **Demand for milk and dairy**: Milk and dairy production increased significantly in India in 2013, indicating a possible increase in the consumption of dairy products in that country.

Overall, these data suggest significant changes in global food production over about half a century, with an expansion of production and greater diversification of crops in response to demographic, economic, and cultural changes.

---

Now let's observe the top 15 feed items by production in 1961 and 2013, highlighting the associated country.

In [40]:
# filter the data to include only the year 1961/2013 and 'Feed' elements
df_1961_Feed = df_fao[(df_fao.index == 1961) & (df_fao['element'] == 'Feed')]
df_2013_Feed = df_fao[(df_fao.index == 2013) & (df_fao['element'] == 'Feed')]

# sort the filtered DataFrame based on the 'production' column in descending order
top_15_production_items_1961_Feed = df_1961_Feed.groupby(['item', 'area']).sum().sort_values(by='production', ascending=False).head(15).reset_index()
top_15_production_items_2013_Feed = df_2013_Feed.groupby(['item', 'area']).sum().sort_values(by='production', ascending=False).head(15).reset_index()

# divide the values on the y-axis by 1000 (thousands)
top_15_production_items_1961_Feed['production'] /= 1000
top_15_production_items_2013_Feed['production'] /= 1000

# get a list of unique areas
all_areas_feed = list(set(top_15_production_items_1961_Feed['area'].unique()).union(set(top_15_production_items_2013_Feed['area'].unique())))

# define colors using Plotly's Pastel1 color scale
colors_custom = px.colors.qualitative.Plotly

# create figure for 1961 Feed items
fig_1961_Feed = go.Figure()

# add traces for each area in 1961
for i, area in enumerate(all_areas_feed):
    data_area = top_15_production_items_1961_Feed[top_15_production_items_1961_Feed['area'] == area]
    fig_1961_Feed.add_trace(go.Bar(x=data_area['item'], y=data_area['production'], name=area, marker_color=colors_custom[i % len(colors_custom)]))

# update layout for 1961 Feed items
fig_1961_Feed.update_layout(
    title=dict(text='Top 15 Feed Items by Production with Respective Areas - Year 1961',x=0.5),
    xaxis=dict(title='Item'),
    yaxis=dict(title='Production (thousands of tons)', ticksuffix='k'),  # add 'k' suffix to tick labels
    barmode='stack',
    width=1800,  # set width to 1800 pixels
    height=400   # set height to 400 pixels
)

# create figure for 2013 Feed items
fig_2013_Feed = go.Figure()

# add traces for each area in 2013
for i, area in enumerate(all_areas_feed):
    data_area = top_15_production_items_2013_Feed[top_15_production_items_2013_Feed['area'] == area]
    fig_2013_Feed.add_trace(go.Bar(x=data_area['item'], y=data_area['production'], name=area, marker_color=colors_custom[i % len(colors_custom)]))

# update layout for 2013 Feed items
fig_2013_Feed.update_layout(
    title=dict(text='Top 15 Feed Items by Production with Respective Areas - Year 2013',x=0.5),
    xaxis=dict(title='Item'),
    yaxis=dict(title='Production (thousands of tons)', ticksuffix='k'),  # add 'k' suffix to tick labels
    barmode='stack',
    width=1800,  # set width to 1800 pixels
    height=400   # set height to 400 pixels
)

# show the plots
fig_1961_Feed.show()
fig_2013_Feed.show()


Several considerations can be made regarding food production in the two years considered:

- **Shift in Production Centers**: In 1961, the United States and Germany were significant producers of feed items like cereals, maize, starchy roots, and potatoes. However, by 2013, China emerged as the leading producer for several feed items, such as cereals, maize, starchy roots, and vegetables.

- **Increased Production in China**: The data show a substantial increase in feed production in China between 1961 and 2013. This growth reflects China's rapid agricultural development and increased demand for feed items due to population growth and changes in dietary preferences.

- **Diversification of Production**: There's evidence of diversification in feed production that suggests a broadening of agricultural activities and shifts in global trade dynamics.

- **Continued Importance of Certain Crops**: Despite changes in production trends, certain crops like cereals, maize, and starchy roots maintained their importance as feed items across both years, indicating their significance in livestock farming and feed industries.

Overall, these observations highlight the dynamic nature of agricultural production, influenced by factors such as technological advancements, changes in consumer preferences, and shifts in global economic landscapes.

---

Reflecting on possible correlations between the production of food and feed:

- **Dietary Trends and Feed Demand**: There could be correlations between dietary trends and the demand for feed. For instance, an increase of vegetables production might coincide with a higher demand for animal feed to support livestock farming and reflecting dietary changes towards more plant-based foods.

- **Crop Interdependencies**: Certain crops used for food production might also serve as key ingredients in animal feed. For example, grains like maize and wheat are not only staple food items but also primary components of animal feed.

- **Livestock Farming and Feed Production**: The expansion of livestock farming operations could drive the demand for feed production. As the population grows and incomes rise, there might be an increased consumption of animal products, leading to higher demand for feed grains and protein sources for livestock.

- **Market Dynamics**: Global market dynamics and trade patterns might influence the production of both food and feed. Changes in demand-supply dynamics, trade agreements, and market prices for agricultural commodities could impact production decisions and investment in food and feed sectors.

- **Environmental Impact**: There could be environmental correlations between food and feed production, such as competition for land and water resources, deforestation for agriculture, and greenhouse gas emissions associated with livestock farming. Efforts to mitigate environmental impacts in one sector could indirectly affect production in the other.

Overall, analyzing correlations between food and feed production requires considerations of various factors including dietary preferences, agricultural practices, market dynamics, and environmental sustainability. Understanding these interrelationships is essential to develop holistic strategies for food security and sustainable agriculture.

---

From the previous plots, it was observed that China is among the largest producers of both food and feed. Let's now examine the trends of the top 3 food and feed products produced by China from 1961 to 2013.

In [41]:
# filter data for China
china_data = df_fao[df_fao['area_code'] == 'CHN']

# define selected food and feed elements
selected_food_elements = ['Vegetables', 'Vegetables, Other', 'Cereals - Excluding Beer']
selected_feed_elements = ['Cereals - Excluding Beer', 'Maize and products', 'Starchy Roots']

# filter data for selected food and feed elements
china_food_data = china_data[(china_data['item'].isin(selected_food_elements)) & (china_data['element'] == 'Food')]
china_feed_data = china_data[(china_data['item'].isin(selected_feed_elements)) & (china_data['element'] == 'Feed')]

# create figure
fig = go.Figure()

# add traces for food items
for item in selected_food_elements:
    fig.add_trace(go.Scatter(x=china_food_data[china_food_data['item'] == item].index,
                             y=china_food_data[china_food_data['item'] == item]['production'],
                             mode='lines',
                             name=item))

# add traces for feed items
for item in selected_feed_elements:
    fig.add_trace(go.Scatter(x=china_feed_data[china_feed_data['item'] == item].index,
                             y=china_feed_data[china_feed_data['item'] == item]['production'],
                             mode='lines',
                             name=item,
                             line=dict(dash='dash')))

# update layout
fig.update_layout(title='Production of Food and Feed 1961-2013 (China)',
                  xaxis=dict(title='Year', tickmode='linear', tick0=df_fao.index.min(), dtick=5, range=[df_fao.index.min() - 3, df_fao.index.max() + 3]),
                  yaxis=dict(title='Production (thousands of tons)'),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                  annotations=[dict(x=0.5, y=-0.1,
                                    text='Continuous lines represent Food; Dashed lines represent Feed',
                                    showarrow=False,
                                    xref='paper',
                                    yref='paper',
                                    font=dict(size=12))],
                  width=1800,  # set width to 1800 pixels
                  height=800,  # set height to 800 pixels
                  title_x=0.5)  # center the title along the x-axis

# show the plot
fig.show()

**Feed Production**: Feed production maintained a relatively stable trend until 2010, suggesting a certain stability in animal feeding practices during that period. However, the observed increase in production after 2010 could indicate a shift in demand for animal feed, new agricultural technologies, or other influences in the agricultural industry.

**Food Production**: Until 1995, cereals constituted the most produced food category, indicating their importance as a primary component of the human diet. However, being surpassed by vegetables suggests a shift in dietary patterns and food preferences. The significant increase in vegetable production since 1990 could be attributed to greater awareness of the health benefits of vegetables, changes in dietary preferences, or agricultural policies aimed at promoting the production of vegetables.

---

Below is an interactive world map to observe and interact with total production of food and feed from 1961 to 2013.

In [42]:
global_prod = df_fao.groupby(['area', 'latitude', 'longitude', 'year'])['production'].sum().reset_index()

fig = px.scatter_geo(
    global_prod,
    lat='latitude',
    lon='longitude',
    color='production',
    color_continuous_scale='turbo',
    size='production',
    size_max=100,
    hover_name='area',
    hover_data={'latitude': False, 'longitude': False},
    animation_frame='year',
    projection='equirectangular',
    range_color=[global_prod['production'].min(), global_prod['production'].max()],
    labels={'production': 'Production (thousands of tons)', 'year': 'Year'}
)

fig.update_layout(
    title='Worldwide total production from 1961 to 2013',
    title_x=0.5,
    title_y=0.95,
    title_xanchor='center',
    title_yanchor='top',
    dragmode=False,
    width=1800,
    height=800
)

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 250
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 80

fig.show()


Now, let's observe the top 10 food items with the highest total emission values, divided by emission source.

In [43]:
# sort by total emissions and select the top 10
df_sorted = df_prod.sort_values(by='total_emissions', ascending=False).head(10)

# calculate the median value of total emissions
median_total_emissions = df_prod['total_emissions'].median()

# select only the desired columns
df_sorted = df_sorted[['food_product', 'land_use_change', 'animal_feed', 'farm', 'processing', 'transport', 'packaging', 'retail']]

# transpose the DataFrame to have food_product as index
df_sorted.set_index('food_product', inplace=True)

# create traces for each emission source
data = []
for column in df_sorted.columns:
    data.append(go.Bar(x=df_sorted.index, y=df_sorted[column], name=column))

# create the layout
layout = go.Layout(title='Top 10 Food Products with Highest Total Emissions',
                   title_x=0.5,  # set title alignment to center
                   xaxis=dict(title='Food Products'),
                   yaxis=dict(title='Emissions (kgCO₂ per kg of product)'),
                   barmode='stack',
                   width=1800,  # set width to 1800 pixels
                   height=800)  # set height to 800 pixels

# create the figure
fig = go.Figure(data=data, layout=layout)

# add a vertical line for the median value
fig.add_shape(
    type="line",
    x0=-0.5,  # position of the vertical line (before the first bar)
    y0=median_total_emissions,
    x1=len(df_sorted.index) + 1,  # position of the vertical line (after the last bar)
    y1=median_total_emissions,
    line=dict(color="black", dash="dash"),
    name=f"Median Total Emissions: {median_total_emissions:.2f}"
)

# Add an annotation for the median
fig.add_annotation(
    x=len(df_sorted.index) - 0.5,  # position x of the annotation (after the last bar)
    y=median_total_emissions,  # position y of the annotation (value of the median)
    text=f"Median Total Emissions: {median_total_emissions:.2f}",
    showarrow=True,
    arrowhead=4,
    ax=100,
    ay=-50
)

# show the plot
fig.show()


- **Beef (beef herd) is by far the most pollutinG**: Beef production is 143% more polluting than the second food (lamb & mutton).

- **Emissions vary depending on the type of product**: Products like beef (both from beef herds and dairy herds), lamb & mutton, and pork have high emissions. Products like coffee and farmed shrimps have relatively lower emissions compared to them. This suggests that some production practices may be more sustainable than others.

- **There is variation in emissions along the supply chain**: Emissions are not only concentrated in primary production but also extend to processing, transportation, packaging, and retail. It is important to consider environmental impact along the entire supply chain.

In general, these conclusions indicate that to reduce the environmental impact of food products, it is important to consider various factors along the entire supply chain, including production methods, transportation, packaging, and distribution. Promoting sustainable agricultural practices and reducing consumption of high-emission products could help mitigate the overall environmental impact of the food sector.

---

Now, let's examine the top 10 food products based on land use.

In [44]:
# sort the DataFrame based on the 'land_use' column in descending order and select the top 10
top_10_land_use = df_prod.sort_values(by='land_use', ascending=False).head(10)

# create a horizontal bar chart
fig = go.Figure()

# add horizontal bars
fig.add_trace(go.Bar(
    y=top_10_land_use['food_product'][::-1],  # reverse the order for a top-down display
    x=top_10_land_use['land_use'][::-1],  # reverse the order for consistency
    orientation='h',  # horizontal orientation
    marker=dict(color=np.linspace(0, 1, len(top_10_land_use)), colorscale='Plasma'),  # color gradient
    text=[],  # empty list to remove numbers on bars
    hoverinfo='text+x',
))

# add a vertical line for the median value
fig.add_shape(
    type="line",
    x0=median_land_use,
    y0=-1,
    x1=median_land_use,
    y1=len(top_10_land_use),  # adjust based on the number of bars
    line=dict(color="black", dash="dash"),
    name=f"Median Land Use: {median_land_use:.2f}"
)

# aggiungi una nota per la mediana
fig.add_annotation(
    x=median_land_use,
    y=len(top_10_land_use)-0.4,  # posizione verticale dell'annotazione
    text=f"Median Land Use: {median_land_use:.2f}",
    showarrow=True,
    arrowhead=4,
    ax=100,
    ay=0
)

# update layout
fig.update_layout(
    title={
        'text': "Top 10 Food Products by Land Use",
        'x': 0.5,  # set title alignment to center
        'y': 0.95  # set title position
    },
    xaxis=dict(title='Land use (m² per kg of product)'),
    yaxis=dict(title='Food Product'),
    width=1800,  # set width to 1800 pixels
    height=800   # set height to 800 pixels
)

# show the plot
fig.show()

- The first two foods for land use (lamb & mutton and beef (beef herd)) use more land compared to the third (cheese) by 320% and 271% more, respectively.This could be due to the need for vast open spaces for livestock grazing.

- Cheese, being animal-derived product, requires a significant amount of land. This might be related to animal husbandry or the production of raw materials needed for its production.

- Olive Oil, Sunflower Oil and coffee require relatively low amounts of land compared to other listed products. This could be due to the ability to cultivate on relatively small plots of land and the need to cultivate in specific geographical areas.

In general, it's noticeable that animal-derived products tend to require more land than plant-based products. Additionally, the amount of land required varies significantly among different types of food products, reflecting the diverse agricultural practices and production requirements associated with each food item.

---

Now let's observe temperature variation over the years for each season and across different continents.

In [45]:
# create a subplot for each season along with its linear regression
seasons = ['Winter', 'Spring', 'Summer', 'Fall']
fig = make_subplots(rows=4, cols=1, subplot_titles=seasons)

# define colors for seasons
colors = {'Winter': 'blue', 'Spring': 'green', 'Summer': 'orange', 'Fall': 'red'}

for i, season in enumerate(seasons, start=1):
    temp_data = df_temp[(df_temp['months'] == season) & (df_temp['country_name'] == 'World')]
    
    # temperature trace
    trace = go.Scatter(x=temp_data.index, y=temp_data['temp'], mode='lines+markers', name=season, line=dict(color=colors[season]))
    fig.add_trace(trace, row=i, col=1)
    
    # linear regression calculation
    X = np.array(temp_data.index).reshape(-1, 1)
    y = np.array(temp_data['temp'])
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    
    # linear regression trace
    trend_line = go.Scatter(x=temp_data.index, y=y_pred, mode='lines', name=f'Trend - {season}', line=dict(color='black', dash='dash'))
    fig.add_trace(trend_line, row=i, col=1)
    
    # add x-axis title for each subplot
    fig.update_xaxes(title_text='Year', row=i, col=1)
    # update y-axes to show ticks from -2 to +2 separated by 0.5 and show only necessary ticks
    fig.update_yaxes(title_text='Temp Change (°C)', row=i, col=1, range=[-0.8, 1.7], dtick=0.5)

# update the layout of the graph
fig.update_layout(
    title='Global Temperature Variation Over Years by Season',
    title_x=0.5,  # center the title
    width=1800,   # total graph width
    height=800,   # total graph height
    legend=dict(orientation='h'),
    hovermode='closest',
)

# show the graph
fig.show()

# list of continents present in the DataFrame
continents = df_temp['country_name'].unique()

fig = make_subplots(rows=len(continents), cols=1, subplot_titles=continents)

for i, continent in enumerate(continents, start=1):
    temp_data = df_temp[(df_temp['months'] == 'Meteorological year') & (df_temp['country_name'] == continent)]
    
    # temperature trace
    trace = go.Scatter(x=temp_data.index, y=temp_data['temp'], mode='lines+markers', name=continent)
    fig.add_trace(trace, row=i, col=1)
    
    # linear regression calculation
    X = np.array(temp_data.index).reshape(-1, 1)
    y = np.array(temp_data['temp'])
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    
    # linear regression trace
    trend_line = go.Scatter(x=temp_data.index, y=y_pred, mode='lines', name=f'Trend - {continent}', line=dict(color='black', dash='dash'))
    fig.add_trace(trend_line, row=i, col=1)
    
    # add x-axis title for each subplot
    fig.update_xaxes(title_text='Year', row=i, col=1)
    # update y-axes to show ticks from -2 to +2 separated by 0.5 and show only necessary ticks
    fig.update_yaxes(title_text='Temp Change (°C)', row=i, col=1, range=[-1.3, 2.4], dtick=0.5)

# update the layout of the graph
fig.update_layout(
    title='Temperature Variation Over Years by Continent',
    title_x=0.5,  # center the title
    width=1800,   # total graph width
    height=1200,  # total graph height
    legend=dict(orientation='h'),
    hovermode='closest',
)

# show the graph
fig.show()

Examining the linear regressions from each plot for every season and continent, and thus overall for the world, a rising trend in temperature variation is observed.

Looking at the plot, it is clear that while summer shows a relatively smooth line, winter exhibits more fluctuations with numerous peak points. This indicates that the effects of global warming are predominantly observed during the winter season. Spring shows similar fluctuations to winter, while fall trends similarly to summer. Moreover, it is evident that the world experienced its warmest winter in 2007.

Regarding continents, Europe demonstrates the most fluctuating trend throughout the period, experiencing the most significant temperature change in 2007.

# Conclusions

In conclusion, the analysis provides profound insights into the trends and dynamics of global food and feed production.

**Trends in Food and Feed Production**: There has been a significant increase in food and feed production over the years, especially in countries like China and India. This trend can be attributed to technological advancements, changes in agricultural practices and population growth.

**Diversification of Production**: There is evidence of greater diversification in food and feed production, reflecting changes in dietary habits and agricultural practices. For example, China's shift from starchy roots to a variety of vegetables, cereals, and fruits indicates a broadening of agricultural activities.

**Changes in Production Centers**: There has been a notable shift in production centers, with China emerging as a major producer for several food and feed items. This reflects the changing global landscape of agricultural production and resource allocation.

**Correlations between Food and Feed Production**: Various factors such as dietary trends, crop interdependencies, livestock farming practices, resource allocation, market dynamics, and environmental considerations influence the production of both food and feed.

**Environmental Impact**: The production of certain food products, particularily animal-derived products, contributes significantly to land use change, greenhouse gas emissions, and other environmental concerns. Understanding these impacts is crucial for developing sustainable agricultural practices.

**Impact on Rising Temperatures**: The intensive production methods used in animal agriculture, coupled with deforestation for pastureland and feed crops, contribute to greenhouse gas emissions and global warming. This has led to more frequent and severe climate-related events, impacting food and feed production.

**Land Requirements**: Animal-derived products generally require more land for production compared to plant-based products. The amount of land required varies among different food products, reflecting the diversity of agricultural practices and production methods.

Overall, the analysis reveals key trends in global food and feed production, highlighting the significant increase in output driven by technology and population growth. The shift towards diversified production, especially in countries like China, reflects changing dietary preferences and agricultural practices.

This evolving landscape underscores the complex correlations between food and feed production, including environmental impacts like greenhouse gas emissions and land use change, particularly from animal-derived products. Urgent action is needed to adopt sustainable practices that mitigate these impacts while ensuring food security amid growing global demand.